# Prototype code 


In [2]:
pip install numpy Pillow ripser persim #Check to make sure the required libraries are installed

Note: you may need to restart the kernel to use updated packages.


In [1]:
#Import all libraries required to execute the script
import os 
import csv
import numpy as np 
from PIL import Image
from ripser import ripser
import matplotlib.pyplot as plt  

#First function to pre-process images (resize them to dimensions 450 x 450 and make binary)
def resize_and_convert_to_binary(input_folder, output_folder_resized, output_folder_bw):
#Creates the directories
    if not os.path.exists(output_folder_resized):
        os.makedirs(output_folder_resized)
    if not os.path.exists(output_folder_bw):
        os.makedirs(output_folder_bw)  

#Loop through the folder for each image
    for filename in os.listdir(input_folder):
#Check each file is saved as .jpg or .jpeg file
        if filename.lower().endswith(('.jpg','.jpeg')):

            try:
#Opens each image
                image_path = os.path.join(input_folder, filename)
                img = Image.open(image_path)
        
#Resize to 450 x 450
                resized_img = img.resize((450,450))
                output_path_resized = os.path.join(output_folder_resized, filename)
                resized_img.save(output_path_resized)
            
#make Binary
                binary_img = resized_img.convert('1')
                output_path_bw = os.path.join(output_folder_bw,f"{os.path.splitext(filename)[0]}_bw.jpg")
                binary_img.save(output_path_bw)
            
#error message if any files fail to process
            except Exception as e:
                print(f"Error processing {filename}:{e}")

#Function to get coordinates and create persistence graphs            
def extract_coordinates_and_generate_diagram(input_folder, output_folder_diagrams,max_coordinates=10000):
    if not os.path.exists(output_folder_diagrams):
        os.makedirs(output_folder_diagrams)

#Loop through each file and check if they are binary
    for filename in os.listdir(input_folder):
        if filename.lower().endswith('_bw.jpg'):
        
#Convert images into arrays using numpy
            try:
                image_path = os.path.join(input_folder, filename)
                binary_img = np.array(Image.open(image_path))

#Get the coordinates of black pixels
                coordinates = np.column_stack(np.where(binary_img == 0))[:max_coordinates]

#Convert the coordinates into a numpy array
                data = np.array(coordinates)

#Compute persistence graphs and save them
                diagrams = ripser(data)['dgms']
                plot_persistence_diagrams(diagrams, output_folder_diagrams, filename)

#Error message if failure
            except Exception as e:
                print(f"Error processing{filename}:{e}")

#Function to create the persistence graphs. Loops through each image and plots the graph then saves them
def plot_persistence_diagrams(diagrams, output_folder, filename):
    for i, diagram in enumerate(diagrams):
        plt.figure(figsize=(6,6))
        plt.title(f"Persistence Diagram {i+1} for {filename}")
        plt.scatter(diagram[:,0], diagram[:,1], c='red', s=50)
        plt.plot([0, max(diagram[:,1])], [0, max(diagram[:, 1])], linestyle='--',color='black')
        plt.xlabel("Birth")
        plt.ylabel("Death")
        plt.savefig(os.path.join(output_folder,
f"{os.path.splitext(filename)[0]}_diagram_{i+1}.png"))
        plt.close()

#Final execution of all segments, check all folders exist
if __name__== "__main__":
    input_folder_raw = "/Users/maxbenson/Desktop/Raw Images"
    output_folder_resized = "/Users/maxbenson/Desktop/Reframed_Images"
    output_folder_bw = "/Users/maxbenson/Desktop/Binary_Images"
    output_folder_diagrams = "/Users/maxbenson/Desktop/Persistence_Graphs"

#Complete all tasks
    resize_and_convert_to_binary(input_folder_raw, output_folder_resized,output_folder_bw)

    extract_coordinates_and_generate_diagram(output_folder_bw, output_folder_diagrams)

KeyboardInterrupt: 